In [4]:
import re
f = open("alldeq14.txt", "r+")
deqsstr = f.read()
deqslist = re.split("\n", deqsstr)

In [5]:
#deqslist= re.findall("deq14.b.*", deqsstr)
#deqslist = '\n'.join(deqssample)
#print(deqssample)
#len(deqssample)

In [6]:
from urllib import request

emailstext = ''

s3_root = "https://flint-text.s3.amazonaws.com/"
for deq in deqslist:       
    url = s3_root + deq
    response = request.urlopen(url)
    raw_text = response.read().decode('utf8')
    emailstext = "".join((raw_text, emailstext)) #changed this line

In [354]:
#print(emailstext) #see full email output

In [358]:
froms = re.findall("From:.*[A-Za-z].*", emailstext)
fromstr = '\n'.join(froms)
def refroms(fromstr):
    fromstr_orgs = re.sub("From:\W","", fromstr) #with (ORG)
    fromstr_noorgs = re.sub("[(].*[)]","", fromstr_orgs)#without (ORG)
    fromstr_nobegpar = re.sub("\s.*[)]","",fromstr_noorgs)
    fromstr6 = re.sub("[A-Za-z]*@.*","",fromstr_nobegpar)
    fromstr_noendpar = re.sub("[(].*","",fromstr6)
    fromstr_noemail1 = re.sub("[<].*[>]*","", fromstr_noendpar)
    fromstr_noemail2 = re.sub("[{].*[}]*","", fromstr_noemail1)
    fromstr_noemail3 = re.sub("\[.*\]*","", fromstr_noemail2)
    fromstr_noquotes = re.sub("\"|\'","",fromstr_noemail3)
    fromstr_nonumbers = re.sub("[0-9]","",fromstr_noquotes)
    fromstr_noendspace1 = re.sub("\s+\n","\n", fromstr_nonumbers)
    fromstr_noendspace2 = re.sub("\s$","",fromstr_noendspace1)
    return fromstr_noendspace2

newfromstr = str(refroms(fromstr))
#print(newfromstr)

In [361]:
ons = re.findall("On.*wro[tf]e", emailstext)
onstr = '\n'.join(ons)
def reons(onstr):
    onstr1 = re.sub("On\s.*at\s[0-9]*:[0-9][0-9]\s[AP]M\W*","", onstr)
    onstr2 = re.sub("On\s.*\s[0-9]*","", onstr1)
    onstr3 = re.sub("[0-9]","",onstr2)
    onstr4 = re.sub("wrote","",onstr3)
    onstr5 = re.sub("[.]","",onstr4)
    onstr6 = re.sub("[A-Za-z]*@.*","",onstr5)
    onstr_noorgs = re.sub("[(].*[)]","", onstr6)
    onstr_nobegpar = re.sub("\s.*[)]","",onstr_noorgs)
    onstr_noendpar = re.sub("[(].*","",onstr_nobegpar)
    onstr_noemail1 = re.sub("[<].*[>]*","", onstr_noendpar)
    onstr_noemail2 = re.sub("\[.*\]*","", onstr_noemail1)
    onstr_noemail3 = re.sub("[{].*[}]*","", onstr_noemail2)
    onstr_noquotes = re.sub("\"|\'","", onstr_noemail3)
    onstr_noendspace1 = re.sub("\s+\n","\n", onstr_noquotes)
    onstr_noendspace2 = re.sub("\s+$","", onstr_noendspace1)
    return onstr_noendspace2
    
newonstr = str(reons(onstr))
#print(newonstr)

In [384]:
roughlist = []
fromsenders = re.split('\n', newfromstr) #splits from str into list
onsenders = re.split('\n',newonstr) #splits on... str into list

#combines the two lists; roughlist contains names found in From: header and On... header
roughlist.extend(onsenders)
roughlist.extend(fromsenders)

roughstr = '\n'.join(roughlist) #makes list into string
print("rough list of " + str(len(roughlist)) + " things that might be names:")
print("- - -")
#print(roughstr)

rough list of 2111 things that might be names:
- - -


In [373]:
ultimatelist = [] #creates final list for final sender names to be put into
ultimatelist.clear()

In [374]:
# extracts sender lines formatted in [Last, First] 
lastcommafirstdude = re.findall("[A-Za-z]+,\s[A-Za-z]+",roughstr)
ultimatelist.extend(lastcommafirstdude) #puts these names into ultimatelist
#print(lastcommafirstdude)
len(lastcommafirstdude) #number of names already in LAST, FIRST format

1653

In [375]:
# extracts sender lines formatted in [First Last] 
firstlast = re.findall("[A-Z][a-z]+.[A-Z][a-z][A-Za-z]*",roughstr)
#print(firstlast)
len(firstlast)

518

In [376]:
#problem: for "Shekter Smith, Liane", "Shekter Smith" is read as a [First Last]
#band-aid solution:
unwanted = {'Shekter Smith', 'Mary Beth','Tracy Jo'}
firstlast1 = [e for e in firstlast if e not in unwanted]
firstlaststring = '\n'.join(firstlast1)
#print(firstlaststring)

In [377]:
#LIST OF ALL NAMES (FIRST LAST FIRST LAST etc.)
allflnames = re.findall('[A-Z][a-z][A-Za-z]+',firstlaststring)
#print(allflnames)
#len(allflnames)

In [386]:
#LIST OF FIRST NAMES
firstnames = re.findall('^[A-Z][a-z]+\s|\n[A-Z][a-z]+\s',firstlaststring)
firstnames1 = ' '.join(firstnames)
firstnames2 = re.sub("\n","",firstnames1)
firstnameslist = re.split("\s+",firstnames2)
firstnameslist.pop()
#print("first names:")
#print(firstnameslist)
#print("")

#LIST OF LAST NAMES
lastnames1 = re.findall('.[A-Z][a-z]+',firstlaststring)
lastnames2 = ''.join(lastnames1)
lastnameslist = re.split('\s',lastnames2)
del lastnameslist[0]
#print("last names:")
#print(lastnameslist)
#print(" ")
#if len(lastnameslist) == len(firstnameslist):
    #print("yay, there are " + str(len(lastnameslist)) + " first and last names.")
#else:
    #print("drat! the number of first names found doesn't match the number of last names found")

In [379]:
print("Reformatted names:")
print("- - -")
for idx, value in enumerate(firstnameslist):
    key = lastnameslist[idx] + ", " + firstnameslist[idx]
    #print(key)
    ultimatelist.append(key) #adds these reformatted names to ultimatelist

Reformatted names:
- - -


In [380]:
roughlist2 = []
roughlist2.extend(lastcommafirstdude)
roughlist2.extend(firstlast1)
#roughlist2 = sum of names found to be in [First Last] and [Last, First]

roughlistd = list(dict.fromkeys(roughlist))
roughlist2d = list(dict.fromkeys(roughlist2))

duplicates = []
weedouts = []
weedouts.clear()
def whatdiff(a, b):
    for i in a:
        if i in b:
            #print(i + " is in borth")
            duplicates.append(i)
    for i in a:
        if i not in b:
            #print(i + " was weeded out")
            weedouts.append(i)
            
msg = len(roughlist) - (len(roughlist2))
if msg > 0:
    n = str(len(roughlist))
    h = str(len(ultimatelist))
    print("out of " + n + " initial items, " + h + " are in ultimatelist.")
    print("")
    whatdiff(roughlistd,roughlist2d)
    print("items that were either weeded out or modified in ultimatelist:" + "\n\n" + str(weedouts))
elif msg == 0:
    print ("No items were weeded out.")
elif msg < 0:
    print ("duplicates present: " + str(duplicates))

out of 2111 initial items, 2063 are in ultimatelist.

items that were either weeded out or modified in ultimatelist:

['Benzle, Richard  _', 'Robert, Carla M.', 'Benzie, Richard  -', 'Devereaux, Tracy Jo', '// To: B//M', 'sarah hulett', 'Michae] Glasgow', 'Fortin, Denise”', 'Michigan Attorney General', 'Brad', 'Stanton, Terry A', 'Prysby, Mike  |', 'Lea Moste |mailtu:|walter', 'Thelen, Mary Beth', 'M/ To: //', 'DEQ-LogLetter', 'Sitko, Anthony M', '- ~ Busch, Stephen', ', Wurfel, Brad', 'Schmitt-Reichert, Anna', 'Hedman, Susan”', 'DEQFOIA', ' Schock, Michael', 'GOV Newsroom', ': Poy, Thomas', 'Howard CTroft', 'Poy, Thomas |mailto:goy.', 'Dykema, Linda D.', 'DEQ-Legislative-Contact', ' . Prysby, Mike', '| Shekter Smith, Liane', 'NgReply', 'Shekter Smith, Llane', 'ZIGMOND KOZICKI', 'Hawes, Mary Kay', 'Wurfel, Brad  Lma_llm.', 'Prysby, Mike  .', 'Deltoral, Miguel «de miguel gov', 'sarah.', '! Shekter Smith, Liane', '// To: /M', 'Michigan Department of Environmental Quality', 'Deltoral, Mig

In [381]:
len(ultimatelist)

2063

In [382]:
print("ULTIMATE LIST of SENDER NAMES in [Last, First]:")
print("- - -")
ultimateliststr = '\n'.join(ultimatelist)
print(ultimateliststr)
# final list :o) 

ULTIMATE LIST of SENDER NAMES in [Last, First]:
- - -
Cook, Pat
Rosenthal, Adam
Rosenthal, Adam
Smith, Liane
Prysby, Mike
Rosenthal, Adam
Wurfel, Brad
Smith, Liane
Wurfel, Brad
Wurfel, Brad
Thelen, Mary
Stanton, Terry
Wurfel, Brad
Wurfel, Brad
Wurfel, Brad
Rosenthal, Adam
Rosenthal, Adam
Smith, Liane
Prysby, Mike
Rosenthal, Adam
Wurfel, Brad
Smith, Liane
Krisztian, George
Thelen, Mary
Krisztian, George
Reynolds, Lawrence
Reynolds, Lawrence
Reynolds, Lawrence
Wurfel, Brad
Erb, Robin
Erb, Robin
Murray, David
Stanton, Terry
Eisner, Jennifer
Kaffer, Nancy
Kaffer, Nancy
Wurfel, Brad
Wurfel, Brad
Burr, Richard
Burr, Richard
Kaffer, Nancy
Wurfel, Brad
Wurfel, Brad
Murray, David
Murray, David
Wurfel, Brad
Henry, James
Wurfel, Brad
Wurfel, Brad
Wurfel, Brad
Feuerstein, Heather
Wurfel, Brad
Holton, Jennifer
Wurfel, Brad
Wurfel, Brad
Wurfel, Brad
Murray, David
Wurfel, Brad
Wurfel, Brad
Rosenthal, Adam
Mack, Melissa
Tommasulo, Karen
Busch, Stephen
Devereaux, Tracy
Clayton, Stacie
Muchmore, Dennis


In [383]:
ultimatelist_senders = list(dict.fromkeys(ultimatelist))

print(str(len(ultimatelist_senders)) + " unique senders found:")
print("")
#print(ultimatelist_senders)
ultimatelist_sendersstr = "\n".join(ultimatelist_senders)
print(ultimatelist_sendersstr)

209 unique senders found:

Casha, Jim
Hill, Bryan
Clayton, Stacie
White, Ed
Dupnack, Jessica
White, Valorie
Brewster, Simone
Holton, Jennifer
Livengood, Chad
Hawes, Mary
Lyon, Nick
Smith, Leslie
Carmody, Stephen
Byrne, Randall
Kosloski, Jessica
Howes, Sarah
FongergLetter, Ronald
Workman, Wayne
Porter, Andrea
Alleman, April
Reynolds, Lawrence
Pluta, Rick
Lesniewski, John
Spangler, Todd
Grijalva, Nancy
Levine, Pete
Graham, Lois
Wisely, John
Maloney, Damon
Lasher, Geralyn
Schock, Michael
Busch, Stephen
Klaver, Christopher
Benzie, Richard
Burgess, Louis
Smith, Lindsey
Biolchini, Amy
Walling, Dayne
Eisner, Jennifer
Smith, Liane
Stuever, Beth
Michigan, Progress
Wyant, Dan
Wurfel, Brad
Wright, Brent
Hansen, Jeff
Averill, James
Pratt, Jeff
Redd, Elaine
Lambrinidou, Yanna
Buhs, Caleb
Butler, Sonya
Brown, Jessica
Deltoral, Miguel
Dickinson, Jordan
Waurfel, Brad
Ragland, Debra
Ockert, Katherine
Tommasulo, Karen
Anderson, Madhu
Egan, Paul
Fortin, Denise
Murray, David
Krisztian, George
Sitko, Antho

In [390]:
froms.clear()
ons.clear()
froms = re.findall("From:.*[A-Za-z].*", emailstext)
ons = re.findall("On.*wro[tf]e", emailstext)
print((str(len(ons) + len(froms))) + " total froms vs. " + str(len(ultimatelist)) + " senders in ultimatelist")
percent = str((len(ultimatelist))/(len(ons) + len(froms))*100)
print(percent + "%")

2154 total froms vs. 2063 senders in ultimatelist
95.7753017641597%


In [389]:
#Wurfel errors
ult = "\n".join(ultimatelist_senders)
re.findall("W.+rfel",ult)

['Wurfel', 'Waurfel', 'Wurfel', 'Whrfel', 'Warfel']